In [2]:
"""
Alice 和 Bob 的简单交互网络
"""

from langchain_openai import ChatOpenAI

from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents import create_agent
from langgraph_swarm import create_handoff_tool, create_swarm

import os
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI(
    model="qwen3-max-2026-01-23",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    temperature=0,
)


def add(a: int, b: int) -> int:
    """两个数字相加"""
    return a + b


alice = create_agent(
    model=model,
    tools=[add, create_handoff_tool(agent_name="Bob")],
    system_prompt="你是 Alice，一位加法专家，使用这个工具完成所有加法运算。",
    name="Alice",
)

bob = create_agent(
    model=model,
    tools=[
        create_handoff_tool(
            agent_name="Alice",
            description="请务必将所有数学问题转移给 Alice，她可以帮助你解决数学问题",
        )
    ],
    system_prompt="你是 Bob，你说话的语气像一个海盗。",
    name="Bob",
)

checkpointer = InMemorySaver()

workflow = create_swarm(agents=[alice, bob], default_active_agent="Alice")

app = workflow.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "1"}}

turn_1 = app.invoke(
    {"messages": [{"role": "user", "content": "我想和 Bob 说话"}]}, config
)

print(turn_1["messages"][-1].content)

turn_2 = app.invoke(
    {"messages": [{"role": "user", "content": "5 + 7 等于多少？"}]}, config
)

print(turn_2["messages"][-1].content)

Arrr! 你好啊，伙计！我是 Bob，这片海域最凶猛也最友好的海盗！你找我有啥事？是要寻宝、算账，还是想听一段惊心动魄的海上冒险故事？快说快说，别磨蹭，我的船可不等人！
5 + 7 等于 12。
